In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy,mean_squared_error, categorical_crossentropy
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import plot_model,to_categorical
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import load_model
from tensorflow.python.framework.graph_util import convert_variables_to_constants
import tensorflow.keras.backend as K

In [ ]:
K.set_learning_phase(0)
model=load_model("../model/model.h5")

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in
                                  tf.compat.v1.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.compat.v1.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                          output_names, freeze_var_names)
    return frozen_graph

In [ ]:
frozen_graph = freeze_session(tf.compat.v1.keras.backend.get_session(),output_names=[out.op.name for out in model.outputs])

In [ ]:
tf.saved_model.save(model, "../model/")

In [ ]:
tf.compat.v1.train.write_graph(frozen_graph, "../model/", "frozen_model.pb", as_text=False)

## optimize

In [ ]:
!python3.7 -m pip install networkx matplotlib pydot graphviz

In [ ]:
model.summary()

In [ ]:
import platform
is_win = 'windows' in platform.platform().lower()

# OpenVINO 2019
if is_win:
    mo_tf_path = '"C:\Program Files (x86)\IntelSWTools\openvino\deployment_tools\model_optimizer\mo_tf.py"'
else:
    # mo_tf.py path in Linux
    mo_tf_path = '/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py'

pb_file = '../model/frozen_model.pb'
output_dir = '../model'
input_shape = [1,235]
input_shape_str = str(input_shape).replace(' ','')
model_name = "sequential"
input_layer = "dense_input"
output_layer = "dense_1/Softmax"

!python3.7 -m pip install networkx
!python3.7 {mo_tf_path} --input_model {pb_file} --output_dir {output_dir} --input {input_layer} --output {output_layer} --input_shape {input_shape_str}  --model_name {model_name} --data_type FP32

In [ ]:
pd.DataFrame(data=X_test).to_csv("input.txt", index=False, header=False, sep='\t')
pd.DataFrame(data=Y_test.argmax(axis=1)).astype(int).to_csv("test.txt", index=False, header=False, sep='\t')

In [ ]:
np.genfromtxt("test.txt", dtype=int)

In [ ]:
model.predict(X_test[:10]).argmax(axis=1)

In [ ]:
Y_test.argmax(axis=1)[:10]

In [ ]:
np.unique(df_labels['primary_site'])

In [ ]:
X_test